# Introdução ao LangChain e aplicações com LLMs



## 1. O que é o Langchain?

* Um framework para desenvolver aplicações que utilizam Large Language Models (LLMs).

* Facilita a integração de modelos, fontes de dados, memória e agentes inteligentes.

* Permite criar chatbots, sistemas de resposta a perguntas, automações de workflow, etc.

Para este estudo, utilizaremos a IA do Google: Gemini
Assim, para fazer o download desta biblioteca, digite no terminal:

```powershell
  pip install langchain-google-genai
```

## 2. Criando pipeline simples

In [ ]:
import os
from dotenv import load_dotenv

# configuração do modelo: mude para a sua chave de API do Google
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
resposta = llm.invoke("Explique a teoria da relatividade em termos simples.") # gerando uma resposta simples
print(resposta.content)

A teoria da relatividade de Einstein é na verdade duas teorias: a **relatividade restrita** e a **relatividade geral**.  Vamos simplificar cada uma:

**Relatividade Restrita (Especial):**

Imagine duas pessoas, Alice e Bob. Alice está parada, e Bob está num foguete voando a uma velocidade muito alta.  A relatividade restrita diz duas coisas importantes:

1. **A velocidade da luz é constante para todos:**  Não importa se você está parado ou movendo-se a uma velocidade incrível, a luz sempre viajará a 299.792.458 metros por segundo em relação a você.  Isso parece estranho, mas é verdade!

2. **Espaço e tempo são relativos:** Para Alice, o tempo passa normalmente. Mas para Bob, no foguete, o tempo passa mais lentamente do que para Alice (de forma imperceptível em velocidades cotidianas, mas significativa em velocidades próximas à da luz).  Além disso, o comprimento do foguete parece menor para Alice do que para Bob.  Ou seja, espaço e tempo se "misturam" e dependem da velocidade relativa 

## 3. Prompts no Langchain 

In [11]:
from langchain.prompts import PromptTemplate

# criando um modelo de prompt
template = PromptTemplate(
    input_variables=["tema"],
    template="Explique sobre {tema} de forma simples e objetiva."
)

# gerando o prompt formatado
prompt_formatado = template.format(tema="Inteligência Artificial")
print(prompt_formatado)


Explique sobre Inteligência Artificial de forma simples e objetiva.


## 4. Cadeias de processamento (Chains)

In [12]:
from langchain.chains import LLMChain

# criando uma cadeia que recebe um prompt e gera uma resposta
chain = LLMChain(llm=llm, prompt=template)

# executando a cadeia
resposta = chain.run("Machine Learning")
print(resposta)


C:\Users\marie\AppData\Local\Temp\ipykernel_9016\1808422957.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=template)
C:\Users\marie\AppData\Local\Temp\ipykernel_9016\1808422957.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = chain.run("Machine Learning")


Machine Learning (Aprendizado de Máquina) é como ensinar um computador a aprender sem ser explicitamente programado para cada tarefa.  Em vez disso, você fornece ao computador muitos exemplos (dados), e ele encontra padrões nesses dados para fazer previsões ou tomar decisões.

Imagine ensinar um cachorro a sentar: você mostra o comando "senta" várias vezes, e o recompensa quando ele faz certo.  O cachorro aprende o padrão entre o comando e a recompensa.  O Machine Learning é similar:  alimentamos o computador com dados, e ele "aprende" os padrões para realizar tarefas como:

* **Classificação:**  Identificar se uma imagem é um gato ou um cachorro.
* **Regressão:** Prever o preço de uma casa baseado em sua área e localização.
* **Agrupamento:**  Agrupar clientes com comportamentos de compra similares.

Em resumo: **dados + algoritmo = aprendizado + previsão/decisão.**


## 5. Memória em aplicações com LLMs

In [13]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# criando memória para armazenar o histórico da conversa
memory = ConversationBufferMemory()

# criando uma cadeia de conversa com memória
chat = ConversationChain(llm=llm, memory=memory)

# testando o chatbot
print(chat.run("Qual é a capital da França?"))
print(chat.run("E qual a população dessa cidade?"))


C:\Users\marie\AppData\Local\Temp\ipykernel_9016\4026064400.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\marie\AppData\Local\Temp\ipykernel_9016\4026064400.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chat = ConversationChain(llm=llm, memory=memory)


A capital da França é Paris.  Paris é uma cidade vibrante e histórica, localizada no norte da França, às margens do rio Sena.  É um importante centro cultural, econômico e político, conhecida por seus monumentos icônicos como a Torre Eiffel, o Louvre e a Catedral de Notre-Dame.  Sua população é de aproximadamente 2,1 milhões de habitantes dentro dos limites da cidade, e a área metropolitana abrange uma população significativamente maior.  A cidade é famosa por sua arquitetura, museus, gastronomia e vida noturna, atraindo milhões de turistas todos os anos.  Seu clima é temperado, com verões quentes e invernos relativamente frios e úmidos.
A população de Paris dentro dos limites da cidade é de aproximadamente 2,1 milhões de habitantes.  No entanto, a área metropolitana de Paris, que inclui os subúrbios, tem uma população significativamente maior, estimada em cerca de 12 milhões de pessoas.  É importante distinguir entre a população da cidade propriamente dita e a população da região metr

## 6. Criando Agentes

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool

# definindo ferramentas que o agente pode usar
def conversor_moeda(tool_input):
    """Recebe uma string no formato 'valor=100, taxa=0.85' e retorna a conversão."""
    try:
        # separar os argumentos baseados no formato esperado "valor=100, taxa=0.85"
        args = dict(item.split("=") for item in tool_input.replace(" ", "").split(","))

        # convertendo para float
        valor = float(args.get("valor", 0))
        taxa = float(args.get("taxa", 1))

        return f"O valor convertido é {valor * taxa:.2f}"
    except Exception as e:
        return f"Erro ao converter: {str(e)}"

# criando a ferramenta
conversor_tool = Tool(
    name="Conversor de Moeda",
    func=conversor_moeda,
    description="Converte um valor de uma moeda para outra com base na taxa de câmbio. "
                "Entrada deve ser no formato 'valor=100, taxa=0.85'."
)

# criando um agente que pode usar ferramentas externas
agent = initialize_agent(
    tools=[conversor_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# testando o agente
resposta = agent.run("Converta 100 dólares para euros considerando uma taxa de 0.85.")
print(resposta)




> Entering new AgentExecutor chain...
Thought: I need to use the Conversor de Moeda tool to convert 100 USD to EUR using the given exchange rate.
Action: Conversor de Moeda
Action Input: valor=100, taxa=0.85
Observation: O valor convertido é 85.00
Thought:Thought: I now know the final answer
Final Answer: 85 euros

> Finished chain.
85 euros


## 7. Deploy de aplicações com Langchain

In [ ]:
from fastapi import FastAPI
from langchain.chains import LLMChain

app = FastAPI()

@app.get("/pergunta/")
def perguntar(pergunta: str):
    resposta = chain.run(pergunta)
    return {"resposta": resposta}

# para rodar a API: uvicorn nome_do_arquivo:app --reload
